# Trailling New LlamaParse + DSPy experimentation below, including:
- Experimentaiton with llamaparase for document parsing.
- customisation of the Retriever, Chain of Thought Modules as well as the signatures. 
- Custom validatoin metric (cosine sim of pred vs actual)
- Dummy data used to compile the language model
- Saving compiler output to be used in RAG system

In [1]:
!pip install llama-index
!pip install pypdf 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.7 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 45.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 KB 50.1 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 62.3 MB/s eta 0:00:000m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 KB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 37.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 KB 8.3 MB/s eta 0:00:

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

In [17]:
from dotenv import load_dotenv
import os

load_dotenv() 
os.getcwd()

'/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry'

In [11]:
!ls /home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data
a

pruhealth-critical-illness-extended-care-iii-product-brochure.pdf
pruhealth-critical-illness-first-protect-ii-product-brochure.pdf


In [22]:

from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio; nest_asyncio.apply()

parser = LlamaParse(
    # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown"  # "markdown" and "text" are available
)

# sync
documents = parser.load_data("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-critical-illness-extended-care-iii-product-brochure.pdf")

# async
# documents = await parser.aload_data("./my_file.pdf")

Started parsing the file under job_id 66781390-0e8d-4d0a-b31b-b198422e4d7f


In [27]:
print(documents[0].text[:1000] + '...')

PRUHealp Critical Illness Extended Care III
Continuous cover for ongoing critical illness – wip lump-sum financial support for cancer every year, up to a total of 660% cover while it persists
Critical Illness Protection Hong Kong Edition
---
## PRUHealth Critical Illness Extended Care III

Now, more than ever, people live with critical illnesses – including cancer. The continuous and extended treatment needed, particularly for cancer, can have a major impact on your family’s finances. PRUHealth Critical Illness Extended Care III offers ongoing protection with limited years of premium payment. Even if you claim for a critical illness, it keeps protecting you with cover for cancer, heart attack and stroke. The plan also aims to give you continuous annual lump-sum financial support throughout your cancer treatment. It also pays a benefit if you have to stay in an intensive care unit (ICU) for health issues beyond critical illnesses, including infectious diseases and injuries.

### Plan hi

In [40]:
print(documents[0].text)

PRUHealp Critical Illness Extended Care III
Continuous cover for ongoing critical illness – wip lump-sum financial support for cancer every year, up to a total of 660% cover while it persists
Critical Illness Protection Hong Kong Edition
---
## PRUHealth Critical Illness Extended Care III

Now, more than ever, people live with critical illnesses – including cancer. The continuous and extended treatment needed, particularly for cancer, can have a major impact on your family’s finances. PRUHealth Critical Illness Extended Care III offers ongoing protection with limited years of premium payment. Even if you claim for a critical illness, it keeps protecting you with cover for cancer, heart attack and stroke. The plan also aims to give you continuous annual lump-sum financial support throughout your cancer treatment. It also pays a benefit if you have to stay in an intensive care unit (ICU) for health issues beyond critical illnesses, including infectious diseases and injuries.

### Plan hi

In [28]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)

In [29]:
nodes = node_parser.get_nodes_from_documents(documents)

17it [00:00, 11230.61it/s]
  0%|          | 0/17 [00:00<?, ?it/s]/home/ryang/.local/lib/python3.10/site-packages/tiktoken/core.py:50: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  self._core_bpe = _tiktoken.CoreBPE(mergeable_ranks, special_tokens, pat_str)
100%|██████████| 17/17 [00:06<00:00,  2.66it/s]


In [30]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [31]:
query_engine = recursive_index.as_query_engine(similarity_top_k=25)

In [33]:
query_1 = "What is treatment sure service? "

response_1 = query_engine.query(query_1)
print(str(response_1))

Treatment Sure service is a comprehensive medical advisory service provided by a designated third-party provider. It includes Global Expert Medical Opinion and Medical Concierge services, offering independent medical opinions, assistance with overseas treatment when necessary, and guidance for non-emergency medical conditions requiring a second opinion.


In [34]:
query_1 = "Why will premium's be changed?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Premiums may be changed due to factors such as claims, investment, and persistency experience.


In [35]:
query_1 = "give me the allocation percentage of the invesment strategy"

response_1 = query_engine.query(query_1)
print(str(response_1))

The allocation percentage of the investment strategy is as follows:
- Fixed-income securities: 55%
- Equity-type securities: 45%


In [36]:
query_1 = "my mum has alzheimers, how does this effect her policy?"

response_1 = query_engine.query(query_1)
print(str(response_1))

If your mum has Alzheimer's, it would not directly impact her policy under the PRUHealth Critical Illness Extended Care III plan. The plan primarily covers specific disease conditions, including Major Disease Conditions and Early Stage Major Disease Conditions such as cancer, heart attack, and stroke. Alzheimer's disease is not explicitly mentioned as one of the covered conditions in the context provided.


In [37]:
query_1 = "Explain the major disease benfits and how it is calculated"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Major Disease Benefits are calculated based on a percentage of the current sum assured, along with any additional face value of Special Bonus if applicable. The benefits include Early Stage Major Disease Benefit, Intensive Care Benefit, Major Disease Benefit, Extended Major Disease Benefit, and Major Disease Benefit under Critical Illness Care Enhancer III. The Early Stage Major Disease Benefit and Intensive Care Benefit are set at 25% of the current sum assured each. The Major Disease Benefit is calculated at 30% of the current sum assured plus the face value of any Special Bonus. The Extended Major Disease Benefit is equivalent to 100% of the current sum assured. Lastly, the Major Disease Benefit under Critical Illness Care Enhancer III is set at 50% of the current sum assured.


In [38]:
query_1 = "Compare Early stage major disease benefit and intensive care benefit"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Early Stage Major Disease Benefit provides a payout of USD 25,000, which is 25% of the current sum assured, for specific early stage major disease conditions. The Intensive Care Benefit offers a payout of USD 25,000, also 25% of the current sum assured, for a Qualified ICU Stay lasting 3 or more consecutive days.


# Repeat but with a prompt - v1

In [41]:

from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio; nest_asyncio.apply()

# parser = LlamaParse(
#     # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
#     result_type="markdown",  # "markdown" and "text" are available
#     parsing_instruction="""
#     This document is an insurance product brochure.

#     When a benefits/coverage/exlusion is describe in the document ammend to it add a text in the follwing benefits string format (where coverage could be an exclusion).

#     For {nameofrisk} and in this condition {whenDoesThecoverageApply} the coverage is {coverageDescription}. 
                                            
#     If the document contain a benefits TABLE that describe coverage amounts, do not ouput it as a table, but instead as a list of benefits string.
                                        
#     """
# )



parser = LlamaParse(
    # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    parsing_instruction="""
    This document is an insurance product brochure.

    At the start of the document, there should be a Title, please return this with one hashtage - eg. #PRUHealth Guardian Critical Illness Plan Series.
    This document name shuold be the product name. 

    For the rest of the document, please look out for headers for sections, usually found at the start of a page, but not every page. 
    Give this one two hashtags. For example - ##The Benefits
    Section titles should only be given if it is a short title like sentence and its strictly on its own line and should encapsulate all the context of the following text chunks, 
    whilst being distinct from the immediate previous text. 

    Within each section, there will be sub-sections, which will be topics of the section. Please return these with 3 hashtags. For example: ###Premium structure. These are topics that
    shouldn't make sense to have their owwn larger sub-section and fit underneath the umbrella of its closeby text chunks. 
    
    Tables are quite complicated in this document. Sometimes columns are merged and sometimes rows are merged. Please take this into consideration when parsing tables. If deemeed necesearry,. 
    Duplicate some values across rows to reflect the merging of specific rows in some columns in the original document.  
    
    """
)

# sync
documents = parser.load_data("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-critical-illness-extended-care-iii-product-brochure.pdf")

# async
# documents = await parser.aload_data("./my_file.pdf")

Started parsing the file under job_id f7655126-d4b0-4705-b0cb-2d1a80c834f7
.

In [42]:
print(documents[0].text)

#PRUHealth Critical Illness Extended Care III

##Critical Illness Protection

###Continuous cover for ongoing critical illness
- Lump-sum financial support for cancer every year
- Up to a total of 660% cover while it persists

Hong Kong Edition
---
#PRUHealth Critical Illness Extended Care III

##Plan Highlights
###Covers 117 disease conditions
###Intensive Care Benefit
- Up to 20% cover for 3 or more consecutive days’ stay in an ICU because of health issues beyond critical illnesses, including infectious diseases and injuries
###Up to 860% coverage against critical illnesses
- Multiple coverage for Cancer, Heart Attack and Stroke after a 100% Major Disease Benefit claim
- Extra cover for new or continued cancer strikes:
- Extended Major Disease Benefit for Cancer – up to 2 extra claims, each with 100% cover
- Cancer Treatment Extra Benefit – up to 2 extra payments of 60% each, starting from 1 year after a previous Cancer claim
- Extended Major Disease Benefit for Heart Attack and Stro

In [43]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)

In [44]:
nodes = node_parser.get_nodes_from_documents(documents)

7it [00:00, 17119.61it/s]
  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:02<00:00,  2.41it/s]


In [45]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [46]:
query_engine = recursive_index.as_query_engine(similarity_top_k=25)

In [49]:
query_1 = "What is treatment sure service? "

response_1 = query_engine.query(query_1)
print(str(response_1))

Treatment Sure service provides the life assured with the option to receive treatment overseas if desired. A specialized Treatment Sure physician case manager assists in selecting suitable specialists based on the medical condition, arranges appointments, offers translation services, and provides support during the recovery phase. Additionally, a dedicated physician case manager who speaks the life assured's language accompanies them during the recovery journey.


In [48]:
query_1 = "Why will premium's be changed?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Premiums may be adjusted due to various factors, including the risk class of the life assured, such as age, gender, and smoking status, as well as the chosen premium term. The insurer has the right to review and modify premium rates for specific risk classes on each policy anniversary, but they will always notify the policyholder in advance before making any changes to the premium.


In [50]:
query_1 = "give me the allocation percentage of the invesment strategy"

response_1 = query_engine.query(query_1)
print(str(response_1))

The allocation percentage of the investment strategy involves a varied mix of investments across different asset types. The current long-term target asset allocation comprises fixed-income securities at 55% and equity-type securities at 45%.


In [51]:
query_1 = "my mum has alzheimers, how does this effect her policy?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Alzheimer's disease is not specifically mentioned in the provided context information regarding the PRUHealth Critical Illness Extended Care III Plan. The plan primarily covers a specific list of critical illnesses and conditions as outlined in the brochure. For any specific details or coverage related to Alzheimer's disease, it would be advisable to refer to the policy documents or contact the insurance provider directly for accurate information on how Alzheimer's disease may impact the policy.


In [52]:
query_1 = "Explain the major disease benfits and how it is calculated"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Major Disease Benefits are calculated based on a percentage of the current sum assured of the PRUHealth Critical Illness Extended Care III plan. The benefit amount will be adjusted to 10% of the current sum assured for a Qualified ICU Stay outside of Hong Kong or Macau. There is a USD 50,000/HKD 400,000 per life limit for the Intensive Care Benefit under all policies with the same benefit for the same life assured. We will not pay any Intensive Care Benefit if the Major Disease Benefit has been paid.


In [53]:
query_1 = "Compare Early stage major disease benefit and intensive care benefit"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Early Stage Major Disease Benefit provides a payout of 25% of the current sum assured, while the Intensive Care Benefit offers a payout of 20% of the current sum assured.


## Parsing nodes into vector db

In [64]:

%pip install llama-index llama-index-vector-stores-qdrant

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [65]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

In [68]:
client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://:"
    # otherwise set Qdrant instance with host and port:
    # host="localhost",
    # port=6333
    # set API KEY for Qdrant Cloud
    # api_key="",
)

In [69]:
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

vector_store = QdrantVectorStore(client=client, collection_name="llama_parse_prudential_health_0")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [70]:
vector_store

QdrantVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=False, collection_name='llama_parse_prudential_health_0', path=None, url=None, api_key=None, batch_size=64, parallel=1, max_retries=3, client_kwargs={}, enable_hybrid=False)

In [74]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "what is the coverage of Carcinoma-in-situ and is there a limit? "
)

In [75]:
print(response)

Carcinoma-in-situ is covered for all organs except for Carcinoma-in-situ of the skin (including melanoma-in-situ). There is a limit of USD 50,000/HKD 400,000 per life for each of the 20 selected Early Stage Major Disease Conditions under all in-force and terminated PRUHealth Critical Illness Extended Care III plan(s) covering the same life assured regardless of where such policy(ies) were issued.


# v2 Prompt - All i have time for

In [180]:

from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio; nest_asyncio.apply()


parser = LlamaParse(
    # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    parsing_instruction="""
    This document is structured as an insurance product brochure.

    Insurance Product Title   
    Identify the document title at the very beginning. The title usually serves as the product name.

    # General Formatting for Headers
    Foridentifying sections within the document:
    - Ensure that the selected section title was on its own line, clearly distinct from both the preceding and following text.
    - Section titles should succinctly summarize the content that immediately follows, capturing all relevant themes or topics discussed within that section.
    - Please don't assign a header if it doesnt relate to a insurance product or key section you'd find in a product brochure.

    ## Sub-section Headers
    Identify sub-sections within each main section. They should be related to tmain section. 

    Parsing Tables
    Handle tables with special attention:
    - Tables may feature merged rows or columns. Accurately replicate data to reflect these merges, ensuring that the table's content remains consistent with the visual presentation in the original document.

    Note: Maintain the logical flow and structure of the document during the parsing process, facilitating an easy-to-navigate and comprehensible data format.

    
    """
)

# sync
documents = parser.load_data("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-critical-illness-extended-care-iii-product-brochure.pdf")

# async
# documents = await parser.aload_data("./my_file.pdf")

Started parsing the file under job_id 13a3e85d-fe78-4536-bf96-016bb83e94a5
.

In [189]:
documents_2 = parser.load_data("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-critical-illness-first-protect-ii-product-brochure.pdf")

Started parsing the file under job_id 7c9eadad-54d4-4761-a471-1c8c03c77078


In [181]:
print(documents[0].text)

# PRUHealth Critical Illness Extended Care III

# PRUHealth Critical Illness Extended Care III

Continuous cover for ongoing critical illness – with lump-sum financial support for cancer every year, up to a total of 660% cover while it persists.

## Critical Illness Protection

Hong Kong Edition
---
# PRUHealth Critical Illness Extended Care III

# PRUHealth Critical Illness Extended Care III

Now, more than ever, people live with critical illnesses – including cancer. The continuous and extended treatment needed, particularly for cancer, can have a major impact on your family’s finances. PRUHealth Critical Illness Extended Care III offers ongoing protection with limited years of premium payment. Even if you claim for a critical illness, it keeps protecting you with cover for cancer, heart attack, and stroke. The plan also aims to give you continuous annual lump-sum financial support throughout your cancer treatment. It also pays a benefit if you have to stay in an intensive care unit 

In [256]:
with open("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-care-iii-parsed.txt", "w") as file:
    file.write(documents[0].text)

In [211]:
print(documents_2[0].text)

# PRUHealth Critical Illness First Protect II

# PRUHealth Critical Illness First Protect II

Affordable, in-depth critical illness cover for your everyday life with protection against 74 disease conditions.

## Critical Illness Protection
---
# PRUHealth Critical Illness First Protect II

# PRUHealth Critical Illness First Protect II

Having the peace of mind from knowing that you and your family’s finances are protected from critical illness is invaluable. PRUHealth Critical Illness First Protect II provides whole-life, lump-sum cover against the financial impact of 74 disease conditions. With affordable premiums over a limited period, the plan gives you in-depth cover. It also pays a benefit if you have to stay in an intensive care unit (ICU) for health issues beyond critical illnesses, including infectious diseases and injuries.

## Plan Highlights

|74 disease conditions|+50%|10yr|
|---|---|---|
|Protects your finances against 74 disease conditions|Intensive Care Benefit – up to 2

In [257]:
with open("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-protect-ii-parsed.txt", "w") as file:
    file.write(documents_2[0].text)

In [ ]:

from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio; nest_asyncio.apply()

# parser = LlamaParse(
#     # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
#     result_type="markdown",  # "markdown" and "text" are available
#     parsing_instruction="""
#     This document is an insurance product brochure.

#     When a benefits/coverage/exlusion is describe in the document ammend to it add a text in the follwing benefits string format (where coverage could be an exclusion).

#     For {nameofrisk} and in this condition {whenDoesThecoverageApply} the coverage is {coverageDescription}. 
                                            
#     If the document contain a benefits TABLE that describe coverage amounts, do not ouput it as a table, but instead as a list of benefits string.
                                        
#     """
# )



parser = LlamaParse(
    # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    parsing_instruction="""
    This document is an insurance product brochure.

    At the start of the document, there should be a Title, please return this with one hashtage - eg. #PRUHealth Guardian Critical Illness Plan Series.
    This document name shuold be the product name. 

    For the rest of the document, please look out for headers for sections, usually found at the start of a page, but not every page. 
    Give this one two hashtags. For example - ##The Benefits
    Section titles should only be given if it is a short title like sentence and its strictly on its own line and should encapsulate all the context of the following text chunks, 
    whilst being distinct from the immediate previous text. 

    Within each section, there will be sub-sections, which will be topics of the section. Please return these with 3 hashtags. For example: ###Premium structure. These are topics that
    shouldn't make sense to have their owwn larger sub-section and fit underneath the umbrella of its closeby text chunks. 
    
    Tables are quite complicated in this document. Sometimes columns are merged and sometimes rows are merged. Please take this into consideration when parsing tables. If deemeed necesearry,. 
    Duplicate some values across rows to reflect the merging of specific rows in some columns in the original document.  
    
    """
)

# sync
documents = parser.load_data("/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry/data/pruhealth-critical-illness-extended-care-iii-product-brochure.pdf")

# async
# documents = await parser.aload_data("./my_file.pdf")

Started parsing the file under job_id f7655126-d4b0-4705-b0cb-2d1a80c834f7
.

In [ ]:
print(documents[0].text)

#PRUHealth Critical Illness Extended Care III

##Critical Illness Protection

###Continuous cover for ongoing critical illness
- Lump-sum financial support for cancer every year
- Up to a total of 660% cover while it persists

Hong Kong Edition
---
#PRUHealth Critical Illness Extended Care III

##Plan Highlights
###Covers 117 disease conditions
###Intensive Care Benefit
- Up to 20% cover for 3 or more consecutive days’ stay in an ICU because of health issues beyond critical illnesses, including infectious diseases and injuries
###Up to 860% coverage against critical illnesses
- Multiple coverage for Cancer, Heart Attack and Stroke after a 100% Major Disease Benefit claim
- Extra cover for new or continued cancer strikes:
- Extended Major Disease Benefit for Cancer – up to 2 extra claims, each with 100% cover
- Cancer Treatment Extra Benefit – up to 2 extra payments of 60% each, starting from 1 year after a previous Cancer claim
- Extended Major Disease Benefit for Heart Attack and Stro

In [ ]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents)

7it [00:00, 17119.61it/s]
  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:02<00:00,  2.41it/s]


In [ ]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [ ]:
query_engine = recursive_index.as_query_engine(similarity_top_k=25)

In [ ]:
query_1 = "What is treatment sure service? "

response_1 = query_engine.query(query_1)
print(str(response_1))

Treatment Sure service provides the life assured with the option to receive treatment overseas if desired. A specialized Treatment Sure physician case manager assists in selecting suitable specialists based on the medical condition, arranges appointments, offers translation services, and provides support during the recovery phase. Additionally, a dedicated physician case manager who speaks the life assured's language accompanies them during the recovery journey.


In [ ]:
query_1 = "Why will premium's be changed?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Premiums may be adjusted due to various factors, including the risk class of the life assured, such as age, gender, and smoking status, as well as the chosen premium term. The insurer has the right to review and modify premium rates for specific risk classes on each policy anniversary, but they will always notify the policyholder in advance before making any changes to the premium.


In [ ]:
query_1 = "give me the allocation percentage of the invesment strategy"

response_1 = query_engine.query(query_1)
print(str(response_1))

The allocation percentage of the investment strategy involves a varied mix of investments across different asset types. The current long-term target asset allocation comprises fixed-income securities at 55% and equity-type securities at 45%.


In [ ]:
query_1 = "my mum has alzheimers, how does this effect her policy?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Alzheimer's disease is not specifically mentioned in the provided context information regarding the PRUHealth Critical Illness Extended Care III Plan. The plan primarily covers a specific list of critical illnesses and conditions as outlined in the brochure. For any specific details or coverage related to Alzheimer's disease, it would be advisable to refer to the policy documents or contact the insurance provider directly for accurate information on how Alzheimer's disease may impact the policy.


In [ ]:
query_1 = "Explain the major disease benfits and how it is calculated"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Major Disease Benefits are calculated based on a percentage of the current sum assured of the PRUHealth Critical Illness Extended Care III plan. The benefit amount will be adjusted to 10% of the current sum assured for a Qualified ICU Stay outside of Hong Kong or Macau. There is a USD 50,000/HKD 400,000 per life limit for the Intensive Care Benefit under all policies with the same benefit for the same life assured. We will not pay any Intensive Care Benefit if the Major Disease Benefit has been paid.


In [ ]:
query_1 = "Compare Early stage major disease benefit and intensive care benefit"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Early Stage Major Disease Benefit provides a payout of 25% of the current sum assured, while the Intensive Care Benefit offers a payout of 20% of the current sum assured.


## Parsing nodes into vector db

In [ ]:

%pip install llama-index llama-index-vector-stores-qdrant

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

In [ ]:
client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://:"
    # otherwise set Qdrant instance with host and port:
    # host="localhost",
    # port=6333
    # set API KEY for Qdrant Cloud
    # api_key="",
)
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

vector_store = QdrantVectorStore(client=client, collection_name="llama_parse_prudential_health_0")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

client_docker = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://:"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="",
)
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

vector_store = QdrantVectorStore(client=client_docker, collection_name="llama_parse_prudential_health_0")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents + ,
    storage_context=storage_context,
)

In [ ]:
vector_store

QdrantVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=False, collection_name='llama_parse_prudential_health_0', path=None, url=None, api_key=None, batch_size=64, parallel=1, max_retries=3, client_kwargs={}, enable_hybrid=False)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "what is the coverage of Carcinoma-in-situ and is there a limit? "
)

In [ ]:
print(response)

Carcinoma-in-situ is covered for all organs except for Carcinoma-in-situ of the skin (including melanoma-in-situ). There is a limit of USD 50,000/HKD 400,000 per life for each of the 20 selected Early Stage Major Disease Conditions under all in-force and terminated PRUHealth Critical Illness Extended Care III plan(s) covering the same life assured regardless of where such policy(ies) were issued.


In [182]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings


# node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)
node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-4-turbo"), num_workers=8)

In [183]:
nodes = node_parser.get_nodes_from_documents(documents)

15it [00:00, 22044.34it/s]
  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:12<00:00,  1.23it/s]


In [184]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [185]:
query_engine = recursive_index.as_query_engine(similarity_top_k=3)

In [186]:
query_1 = "What is treatment sure service? "

response_1 = query_engine.query(query_1)
print(str(response_1))

Treatment Sure service is a one-stop professional medical advisory service provided by a designated third-party service provider. It includes Global Expert Medical Opinion and Medical Concierge services. The service is designed for non-emergency medical conditions that require a second opinion, such as cancer, gastroenteric diseases, and orthopaedic issues. The service offers independent medical opinions, facilitates overseas treatment if necessary, and provides personalized medical advice from a network of over 50,000 global medical experts in 450+ specialties.


In [187]:
query_1 = "Why will premium's be changed?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Premiums may be changed due to factors such as the insured individual's age, the selected premium term, the currency option chosen, any changes in the coverage or benefits provided by the insurance policy, or adjustments based on the insurer's underwriting criteria or financial considerations.


In [188]:
query_1 = "give me the allocation percentage of the invesment strategy"

response_1 = query_engine.query(query_1)
print(str(response_1))

The allocation percentage of the investment strategy includes a mix of equity-type securities and fixed-income securities.


In [84]:
query_1 = "my mum has alzheimers, how does this effect her policy?"

response_1 = query_engine.query(query_1)
print(str(response_1))

Alzheimer's disease is not specifically mentioned in the provided context information regarding the PRUHealth Critical Illness Extended Care III Plan. The plan primarily covers a specific list of critical illnesses and conditions as outlined in the details provided. Therefore, the presence of Alzheimer's disease would not directly impact the policy based on the information available.


In [85]:
query_1 = "Explain the major disease benfits and how it is calculated"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Major Disease Benefits are calculated based on the current sum assured of the PRUHealth Critical Illness Extended Care III plan. The Extended Major Disease Benefit provides coverage for major diseases up to a certain age and requires specific conditions to be fulfilled for claims to be made. The Cancer Treatment Extra Benefit is also part of the coverage and is paid after the Major Disease Benefit or Extended Major Disease Benefit for Cancer has been claimed. The benefits are structured to provide financial support in case of major diseases, with specific claim limits and conditions outlined in the plan.


In [86]:
query_1 = "Compare Early stage major disease benefit and intensive care benefit"

response_1 = query_engine.query(query_1)
print(str(response_1))

The Early Stage Major Disease Benefit provides coverage for specific early stage major disease conditions, allowing for claims up to a maximum of 3 times for different conditions. The benefit amount is based on the diagnosed condition and can be claimed multiple times within the specified limits.

On the other hand, the Intensive Care Benefit offers coverage for ICU treatment if the life assured needs to stay in the ICU for 3 or more consecutive days. This benefit pays up to 20% of the current sum assured of the plan as a one-time payment for ICU treatment.

In summary, the Early Stage Major Disease Benefit caters to early stage major disease conditions with multiple claim opportunities, while the Intensive Care Benefit provides coverage specifically for ICU treatment with a one-time payment.


## Parsing nodes into vector db

In [ ]:

%pip install llama-index llama-index-vector-stores-qdrant

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [87]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

In [251]:
q_client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://:"
    # otherwise set Qdrant instance with host and port:
    # host="localhost",
    # port=6333
    # set API KEY for Qdrant Cloud
    # api_key="",
)

In [250]:
q_client_2 = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://:"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="",
)

In [159]:
len(documents + documents_2)

2

In [190]:
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

vector_store = QdrantVectorStore(client=q_client, collection_name="llama_parse_prudential_health_both_pdfs")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents = documents + documents_2,
    storage_context=storage_context,
    show_progress=True
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/27 [00:00<?, ?it/s]

In [253]:
index.storage_context.persist()

In [192]:
storage_context_ = StorageContext.from_defaults(persist_dir="./storage")
index_ = load_index_from_storage(storage_context)

In [252]:
storage_context_ = StorageContext.from_defaults(persist_dir="./storage")
index_ = load_index_from_storage(storage_context)

In [193]:
query_engine = index_.as_query_engine()
response = query_engine.query(
    "what is the coverage of Carcinoma-in-situ and is there a limit? "
)
print(response)

Carcinoma-in-situ is covered for all organs except for Carcinoma-in-situ of the skin. There is a limit of USD 50,000/HKD 400,000 per life for each of the selected Early Stage Major Disease Conditions under all in-force and terminated PRUHealth Critical Illness Extended Care III plans covering the same life assured, regardless of where the policies were issued.


# Continue with DSPY Using qDrant

In [195]:
import openai
import dspy
import requests
from typing import List, Union, Optional

from qdrant_client import QdrantClient
# q_client = QdrantClient(":memory:")
openai_client = openai.Client()
collection_name = "llama_parse_prudential_health_both_pdfs"

In [249]:
q_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='llama_parse_prudential_health_0'), CollectionDescription(name='llama_parse_prudential_health_both_pdfs')])

In [196]:
# Custom Retrieval Model class
class DSPythonicRMClientQdrantCustom(dspy.Retrieve):
    def __init__(self, q_client, collection_name: str,embedding_model ,k: int = 3):
        super().__init__(k=k)
        self.q_client = q_client
        self.collection_name = collection_name
        self.embedding_model = embedding_model

    def forward(self, query: str, k: Optional[int] = None) -> dspy.Prediction:
        query_embedding = openai_client.embeddings.create(
            input=[query],
            model=self.embedding_model
        ).data[0].embedding
        
        results = self.q_client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            limit=k if k else self.k
        )

        import json 

        # print(json.loads(results[0].payload["_node_content"])["text"])
        # return dspy.Prediction(passages=[point.payload['text'] for point in results])
        return dspy.Prediction(passages=[json.loads(results[0].payload["_node_content"])["text"] for point in results])

# Example use in a DSPy Pipeline
class DSPyPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.rm = DSPythonicRMClientQdrantCustom(q_client, collection_name = collection_name,embedding_model="text-embedding-3-small", k=3)

    def forward(self, query: str):
        passages = self.rm(query).passages
        return passages

# Testing the pipeline
if __name__ == "__main__":
    pipeline = DSPyPipeline()
    query = "If i get cancer twice, what happens?"
    results = pipeline.forward(query)
    print("Retrieved Passages:", results)

Retrieved Passages: ["We will pay 100% of the current sum assured each time as an Extended Major Disease Benefit, up to a total of 400% of the current sum assured of your PRUHealth Critical Illness Extended Care III.\n\n### Multiple Claims on Major Disease Conditions\n\nFor multiple claims on Major Disease Conditions before age 86, the benefit payout can reach up to 500% protection. There is a waiting period for the Extended Major Disease Benefit, with specific conditions for subsequent Cancer claims under the Extended Major Disease Benefit.\n---\n# PRUHealth Critical Illness Extended Care III: Cancer Coverage\n\n# A stronger safety net for Cancer\n\n## Cancer Recurrence Coverage\n\nWhat happens if the Cancer unfortunately recurs or persists during the 3-year waiting period for Cancer? As soon as 1 year after the date of diagnosis of each preceding Cancer which we have paid the Major Disease Benefit or Extended Major Disease Benefit, we will offer the Cancer Treatment Extra Benefit.\n\

In [197]:
import os
import sys
import dspy
import pkg_resources
from dspy import Signature, InputField, OutputField, Module, Predict, Prediction
from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import answer_exact_match, answer_passage_match
from dspy import Example

In [198]:
# Update DSPy settings to use this RM
# dspy.settings.configure(rm=dspythonic_rm)


import dspy
llm = dspy.OpenAI(model='gpt-4-turbo')
qdrant_retriever_model = DSPythonicRMClientQdrantCustom(q_client,
                                                        
                                                        collection_name = collection_name, 
                                                        embedding_model="text-embedding-3-small", 
                                                        k=5)
dspy.settings.configure(lm=llm, rm=qdrant_retriever_model)

class GenerateAnswer(dspy.Signature):
    """Answer questions with descriptive answers, you are outputting to customers
        of an insurance company, so helpfully answering the question is essential. 
    """

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="""make sure the customer's query is 1. grounded in the response and appropriately 
                              answered and would be happy to have read your accurate and comprehensive response. Don't be over 3 sentences though
                              If the answer is not grounded in the context, and you are not sure <30% confident in your answer, 
                              please go with your intuition on your pre-trained knowledge to answer and let the user know the most likely product it is found in eg- PRUHealth Guardian 
Critical Illness Plan Series""")
    print("Class 1 created")

class RAG(dspy.Module):
    def __init__(self, num_passages=5):
        super().__init__()

        self.retrieve = DSPythonicRMClientQdrantCustom(
            q_client=q_client,
            collection_name=collection_name,
            embedding_model="text-embedding-3-small",
            k=num_passages
        )

        # self.generate_answer = dspy.ChainOfThought("context, question -> answer")
        self.generate_answer = Predict(GenerateAnswer)
        print("Class 2 created")

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)


Class 1 created


In [199]:
uncompiled_rag = RAG()

example_query = "what is the death benfit"

response = uncompiled_rag(example_query)

print(response.answer)


Class 2 created


Context: The insurance plan discussed includes various benefits and exclusions related to the life assured's coverage under the PRUHealth Critical Illness Extended Care III and Critical Illness Care Enhancer III.

Question: What is the death benefit?

Answer: The death benefit in the context of the PRUHealth Critical Illness Extended Care III is typically a payout made to the beneficiaries of the policy upon the death of the life assured. However, specific details about the death benefit are not explicitly outlined in the provided excerpts of the insurance product brochure. For precise information regarding the death benefit, including any conditions or exclusions that may apply, it is recommended to refer to the full policy documentation or contact customer support directly.


In [213]:
uncompiled_rag = RAG()

example_query = "What is First protect II? "

response = uncompiled_rag(example_query)

print(response.answer)


Class 2 created
Context: PRUHealth Critical Illness: First Protect II is a comprehensive insurance plan designed to offer financial protection against critical illnesses.

Question: What is First Protect II?

Answer: PRUHealth Critical Illness First Protect II is a whole-life insurance plan that provides lump-sum coverage against the financial impact of 74 disease conditions, including both major and early-stage diseases. It features affordable premiums, intensive care benefits, and additional coverage options such as a premium waiver after an early stage major disease claim and a Benefit Protector Option to combat inflation. This plan is designed to offer both protection and long-term savings, ensuring peace of mind for you and your family.


In [214]:
uncompiled_rag = RAG()

example_query = "Explain both PruHealth Critical illness extended Care III and the First protect II? "

response = uncompiled_rag(example_query)

print(response.answer)


Class 2 created


Context: The provided information details the PRUHealth Critical Illness First Protect II plan but does not mention a plan specifically named "PRUHealth Critical Illness Extended Care III."

Question: Explain both PruHealth Critical illness extended Care III and the First protect II?

Answer: The PRUHealth Critical Illness First Protect II offers comprehensive coverage for 74 disease conditions, including major and early-stage diseases, with benefits like ICU stays for non-critical conditions, a premium waiver after an early-stage disease claim, and options to combat inflation. Unfortunately, there is no information provided on a plan named "PRUHealth Critical Illness Extended Care III" in the provided context. For accurate details on this specific plan, I recommend checking the latest product bro


## Extend to Multi-Hop RAG

In [227]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question.
        If part of the query is answered, try focus on the part that isn't with a completely differently framed query
    """

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

In [228]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        # self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.retrieve = DSPythonicRMClientQdrantCustom(q_client, collection_name = collection_name,
                                                embedding_model="text-embedding-3-small", 
                                                 k=passages_per_hop)

        # self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.generate_answer = Predict(GenerateAnswer)

        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [231]:
# Ask any question you like to this simple RAG program.
# my_question = "What is PRUHealth Critical Illness First Protect II?"
my_question = "Explain both PruHealth Critical illness extended Care III and the PRUHealth Critical Illness First Protect II ? "

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen(passages_per_hop=3, max_hops=2)  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Explain both PruHealth Critical illness extended Care III and the PRUHealth Critical Illness First Protect II ? 
Predicted Answer: Context: The provided information outlines the features and benefits of two distinct insurance plans offered by PruHealth: the PRUHealth Critical Illness Extended Care III and the PRUHealth Critical Illness First Protect II.

Question: Explain both PruHealth Critical illness extended Care III and the PRUHealth Critical Illness First Protect II?

Answer: The PRUHealth Critical Illness First Protect II is designed to offer comprehensive protection against 74 disease conditions, including major and early-stage diseases. It provides a lump-sum payment for major disease conditions and offers additional benefits such as a 50% extra protection for major disease or death claims within the first 10 years, and an Intensive Care Benefit for ICU stays due to severe health issues. On the other hand
Retrieved Contexts (truncated): ['# PRUHealth Critical Illness

In [229]:
# Ask any question you like to this simple RAG program.
# my_question = "What is PRUHealth Critical Illness First Protect II?"
my_question = "Explain both PruHealth Critical illness extended Care III and the PRUHealth Critical Illness First Protect II ? "

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen(passages_per_hop=5, max_hops=3)  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Explain both PruHealth Critical illness extended Care III and the PRUHealth Critical Illness First Protect II ? 
Predicted Answer: Context: PRUHealth Critical Illness Extended Care III and PRUHealth Critical Illness First Protect II are two distinct insurance products offered by Prudential, designed to provide financial protection against critical illnesses.

Question: Explain both PRUHealth Critical Illness Extended Care III and the PRUHealth Critical Illness First Protect II?

Answer: PRUHealth Critical Illness First Protect II offers comprehensive, whole-life coverage against 74 disease conditions, including major and early-stage diseases. It provides a lump-sum payment for major disease claims, a premium waiver following an early-stage disease claim, and an Intensive Care Benefit for ICU stays due to severe health issues. In contrast, PRUHealth Critical Illness Extended Care III extends coverage beyond the initial critical
Retrieved Contexts (truncated): ['# PRUHealth Cri

In [226]:
len(pred.context)

1

# Optimization Logic

In [145]:
# Create validation logic 
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = answer_exact_match(example, pred)
    answer_PM = answer_passage_match(example, pred)
    return answer_EM and answer_PM

In [233]:
import openai
import numpy as np
import numpy as np

def cosine_similarity(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Function to get embeddings from the OpenAI API
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai_client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# Calculate cosine similarity using OpenAI embeddings
def validate_answer_cosine_sim(example, pred, trace=None):
    # Assuming `example` and `pred` have a property `answer` containing the text
    example_embedding = get_embeddings(example.answer)
    pred_embedding = get_embeddings(pred.answer)
    
    return cosine_similarity(np.array(example_embedding), np.array(pred_embedding))


In [234]:
# Example usage
class TextObject:
    def __init__(self, answer):
        self.answer = answer

example = TextObject("What is machine learning?")
pred = TextObject("Machine learning involves computers learning from data.")

# Get the similarity score
similarity_score = validate_answer_cosine_sim(example, pred)
print(f"Cosine Similarity: {similarity_score}")

Cosine Similarity: 0.6513769201916587


In [235]:
# PRUHealth Critical Illness Extended Care III
train_example1 = Example(question="What is the total coverage percentage I can receive for critical illnesses under the PRUHealth Critical Illness Extended Care III?", 
                         answer="Under the PRUHealth Critical Illness Extended Care III, you can receive up to 860% coverage against critical illnesses. This includes multiple claims for conditions like cancer, heart attack, and stroke, up to a specified age limit.")
train_example2 = Example(question="Can the Critical Illness Care Enhancer III be converted to a new life policy in PRUHealth Critical Illness Extended Care III?", 
                         answer="Yes, the Critical Illness Care Enhancer III can be converted into a new cash-value life policy. Conditions for conversion include no prior claims under the policy except Parental Premium Waiver Benefit, conversion before the life assured's 66th birthday, application within 1 month of the Enhancer's term end, and adherence to specified terms and minimum sum assured requirements.")
train_example3 = Example(question="Can you explain the conditions and waiting periods for making claims for the Extended Major Disease Benefit, particularly for cancer?", 
                         answer="For cancer, the waiting period between subsequent claims for the Extended Major Disease Benefit is at least one year from the diagnosis date of the last claim and three years if both claims are for cancer. The life assured must also survive for at least 14 days after diagnosis to be eligible for the benefit.")
train_example4 = Example(question="What specific diseases are covered under the Early Stage Major Disease Conditions in this policy?", 
                         answer="The policy covers various early stage conditions, including Carcinoma-in-situ, early thyroid or prostate cancer, and less aggressive malignancies, among others. Each condition has specific coverage limits and conditions for claiming benefits. For a full list please refer to the Extended Care III brochure")
train_example5 = Example(question="In PRUHealth Critical Illness Extended Care III, If I am diagnosed with a major disease, does the policy cover me for subsequent conditions like heart attack or stroke, and up to what age?", 
                         answer="The policy continues to cover the insured for subsequent major diseases like heart attack or stroke after an initial claim, up to the age of 86. Each subsequent condition allows for additional claims under the Extended Major Disease Benefit.")


# PRUHealth Critical Illness First Protect II
train_example6 = Example(question="What is the guaranteed cash value of the PRUHealth Critical Illness First Protect II?", 
                         answer="The plan pays a guaranteed cash value if the policy is surrendered on or after the third policy anniversary, plus any non-guaranteed one-off bonuses that may apply from the fifth policy anniversary.")
train_example7 = Example(question="For PRUHealth Critical Illness First Protect II?, Can I claim more than once for Early Stage Major Disease Conditions?", 
                         answer="Yes, the Early Stage Major Disease Benefit allows for claims up to three times. Conditions like Carcinoma-in-situ and Coronary Angioplasty can each be claimed up to two times, each for up to 25% of the sum assured, while other conditions can be claimed once for up to 20% of the sum assured.")
train_example8 = Example(question="Is there a premium waiver available in this plan for PRUHealth Critical Illness First Protect II", 
                         answer="Yes, there is a 12-month premium waiver after an early stage major disease claim, which eases the financial burden by waiving the premium payments due in the next 12 months following the claim.")
train_example9 = Example(question="What happens to the coverage if the life assured is diagnosed with a recurrent condition, according to First Protect II?", 
                         answer="For recurring conditions like cancer, if diagnosed again, the policy allows for subsequent claims under the major disease or early stage disease benefits, subject to the terms of the plan and remaining sum assured.")
train_example10 = Example(question="Are there any exclusions or conditions where the policy will not pay out?", 
                          answer="Yes, exclusions include pre-existing conditions, conditions diagnosed within 90 days of policy start or reinstatement, self-inflicted injuries, and conditions caused by war or participation in criminal activities. The policy also will not pay for ICU stays that arise from situations like cosmetic treatments not caused by accidents, or stays primarily for investigation or physiotherapy")

# PRUHealth Critical Illness Extended Care III  and  PRUHealth Critical Illness First Protect II
train_example11 = Example(question="How do the subsequent claims for critical illnesses differ between PRUHealth Critical Illness Extended Care III and PRUHealth Critical Illness First Protect II?", 
                         answer="The PRUHealth Critical Illness Extended Care III allows for multiple subsequent claims for the same critical illnesses like cancer, heart attack, and stroke, even after an initial major disease claim, with up to 860% total coverage potential. In contrast, PRUHealth Critical Illness First Protect II does not explicitly detail the allowance for multiple claims for the same illness beyond the initial and early stage claims.")
train_example12 = Example(question="What are the differences in the Intensive Care Benefit when the insured is outside Hong Kong for PRUHealth Critical Illness Extended Care III and PRUHealth Critical Illness First Protect II?", 
                         answer="Both plans reduce the ICU benefit for hospital stays outside Hong Kong. However, Extended Care III adjusts the cover to up to 10% of the current sum assured, while First Protect II also adjusts to 10% but caps the benefit with a specific limit under all policies for the same life assured, reflecting a structured limitation not detailed in Extended Care III.")
train_example13 = Example(question="How does the death benefit compare between PRUHealth Critical Illness First Protect II and PRUHealth Critical Illness Extended Care III?", 
                         answer="In PRUHealth Critical Illness First Protect II, the death benefit includes 100% of the current sum assured plus the face value of any applicable bonuses, and it explicitly deducts any claims made for early stage or major disease benefits. In contrast, PRUHealth Critical Illness Extended Care III generally provides continuing coverage up to a certain percentage of the sum assured after major illness claims, with specifics depending more on the sequence and type of claim rather than a direct deduction method.")
train_example14 = Example(question="How do the bonus structures differ in PRUHealth Critical Illness Extended Care III compared to PRUHealth Critical Illness First Protect II?", 
                         answer="PRUHealth Critical Illness First Protect II offers a Terminal Bonus that is a one-off, non-guaranteed bonus declared after the fifth policy anniversary, which does not accumulate within the policy. The Extended Care III plan does not explicitly mention a similar terminal bonus but focuses on multiple, layered benefits that can total up to 860% of the sum assured for critical illnesses, suggesting a different structure in how benefits accrue and are utilized over time.")
train_example15 = Example(question="What is the impact of the Benefit Protector Option in PRUHealth Critical Illness First Protect II compared to similar options in PRUHealth Critical Illness Extended Care III?", 
                         answer="The Benefit Protector Option in First Protect II allows for an automatic increase in protection each year to combat inflation, potentially increasing the sum assured by up to 200% over time. In Extended Care III, there isn’t a similar explicitly named feature described in the documentation provided, suggesting that inflation protection may need to be managed differently or may not be as prominently featured as in the First Protect II plan.")
trainset = [
    train_example1.with_inputs('question'),
    train_example2.with_inputs('question'),
    train_example3.with_inputs('question'),

    train_example6.with_inputs('question'),
    train_example7.with_inputs('question'),
    train_example8.with_inputs('question'),

    train_example11.with_inputs('question'),
    train_example12.with_inputs('question'),
    train_example13.with_inputs('question'),
]

devset = [
    train_example4.with_inputs('question'),
    train_example5.with_inputs('question'),

    train_example9.with_inputs('question'),
    train_example10.with_inputs('question'),

    train_example14.with_inputs('question'),
    train_example15.with_inputs('question'),

]


print("Training and Test Sets created")

Training and Test Sets created


In [236]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=validate_answer_cosine_sim)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)

 44%|████▍     | 4/9 [01:11<01:29, 17.97s/it]

Bootstrapped 4 full traces after 5 examples in round 0.


# Saving the optimised program! 

In [242]:
os.getcwd()

'/home/ryang/Projects/llm-chatbot-rag/qdrant_openai/qdrant_openai_ry'

In [243]:
compiled_baleen.save("dspy_program/compiled.txt")

In [244]:
loaded_program = SimplifiedBaleen()
loaded_program.load(path="dspy_program/compiled.txt")

In [239]:
%pip install --upgrade jinja2

Defaulting to user installation because normal site-packages is not writeable
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
Note: you may need to restart the kernel to use updated packages.


In [237]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_dev = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

uncompiled_baleen_retrieval_score = evaluate_on_dev(uncompiled_baleen, metric=validate_answer_cosine_sim, display=False)



/home/ryang/.local/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
Average Metric: 4.939394779767698 / 6  (82.3): 100%|██████████| 6/6 [01:30<00:00, 15.04s/it] 


Average Metric: 4.939394779767698 / 6  (82.3%)


ImportError: Pandas requires version '3.1.2' or newer of 'jinja2' (version '3.0.3' currently installed).

In [240]:
compiled_baleen_retrieval_score = evaluate_on_dev(compiled_baleen, metric=validate_answer_cosine_sim)

print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

Average Metric: 4.939394779767698 / 6  (82.3): 100%|██████████| 6/6 [00:09<00:00,  1.54s/it] 
/home/ryang/.local/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 4.939394779767698 / 6  (82.3%)


ImportError: Pandas requires version '3.1.2' or newer of 'jinja2' (version '3.0.3' currently installed).

In [241]:

print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

## Retrieval Score for uncompiled Baleen: 75.08


NameError: name 'compiled_baleen_retrieval_score' is not defined